# **NYPD Nefret Suçları Analizi**

Bu proje, **NYPD**'deki suçlar üzerine çeşitli faktörlerin analizini içeriyor.
## **Veri Kümesi Hakkında**
Veri kümesi önceden temizlenmemiştir.**NYPD** tarafından gerçekleştirilen **Nefret Suçları** listesini içerir. `13` özellikten oluşur. Bunlardan bazıları:
- **Tam Şikayet Kimliği (Full Complaint ID):** Benzersiz bir şikayet kimliği sağlar.
- **Şikayet Yıl Numarası (Complaint Year Number):** Şikayet yılını sağlar.
- **Ay Numarası (Month Number):** Şikayet ayını sağlar.
- **Kayıt Oluşturma Tarihi (Record Create Date):** Kayıt oluşturma tarihini sağlar.
- **Şikayet Polis Karakolu Kodu (Complaint Precinct Code):** Şikayet karakolu kodunu sağlar.

NYPD nefret suçu veri analizi, bu olayların zamansal desenlerini, zaman içindeki değişiklikler gibi, ve coğrafi dağılımını, nefret suçlarının daha yüksek yoğunlukta olduğu alanları belirleyerek incelemektedir. Suçların ayrıntılarını incelemek, suç türleri, ilişkili kanun kodları, suçların arkasındaki önyargı nedenleri ve tutuklama eğilimleri gibi, bu olayların doğasını ve polisin nasıl yanıt verdiğini daha derinlemesine anlamamızı sağlar.

# **Kütüphanelerin ve Verinin İçe Aktarılması**
---
Burada, projemiz için gerekli tüm `kütüphaneleri` ve tüm gerekli `kaynak dosyaları` içe aktarılacaktır.

In [ ]:
import numpy as np      
import pandas as pd     
import seaborn as sns   
import matplotlib.pyplot as plt 

import warnings
warnings.filterwarnings("ignore")


df = pd.read_csv("/kaggle/input/nypd-hate-crimes/nypd_hate_crimes.csv")
df.head()


Korelasyon Matrisi

Bu verinin analiz edilmeden önce daha fazla `temizlenmesi` ve `işlenmesi` gerekmektedir.

# **Veri Temizleme**
---
Bu bölümde, tüm `boş değerleri` uygun önlemlerle temizliyoruz.

In [ ]:
# sütun bilgisine bakma
df.info()

Toplam 2406 satır ve 13 sütun bulunmaktadır.

NOT: **Tutuklama Tarihi (Arrest Date)** ve **Tutuklama Kimliği (Arrest Id)** sütunları çok düşük veri noktasına sahip olduğundan kaldırılmalıdır.


# **Veri Ön İşleme**
---

## **1. Yalnızca ilgili sütunların seçilmesi**
Burada, yalnızca gerekli sütunları seçiyoruz.

In [ ]:
df.columns
df = df[['Record Create Date', 'Complaint Precinct Code', 'Patrol Borough Name', 'County', 'Law Code Category Description', 'Offense Description',
       'Bias Motive Description', 'Offense Category']]
df.head()

## **2. Tarih-saat verilerinin çıkarılması**
Verilen tarih-saat verilerinden Tarih, Ay ve Yıl bilgilerini çıkarıyoruz.


In [ ]:
df.head()
# tarihi tarih-zaman biçimine dönüştürme
df['record_date'] = pd.to_datetime(df['Record Create Date'], format='mixed')

# haftanın gününü, tarihi, ayı ve yılı ayrı sütunlara ayırma
df['date'] = df['record_date'].dt.day
df['day'] = df['record_date'].dt.strftime('%a')
df['month'] = df['record_date'].dt.strftime('%b')
df['year'] = df['record_date'].dt.year
df.head()
# özellik listesini sıralama
df = df[['date', 'month', 'year', 'day', 'record_date', 'Patrol Borough Name', 'County', 'Law Code Category Description',
         'Bias Motive Description', 'Offense Category', 'Complaint Precinct Code']]

df.head()


## **3. Sütunların Yeniden Adlandırılması**
Sütun adlarını daha basit bir şeye yeniden adlandırıyoruz.

In [ ]:
# mevcut sütun adlarına bakma
df.columns
# sütunları yeniden adlandırma
df.columns = ['date', 'month', 'year', 'day', 'record_date', 'patrol', 'county', 'crime', 'motive', 'offense', 'precinct']
df.head()


## **4. Verinin Yeniden Düzenlenmesi**
Veriyi biraz daha anlaşılır hale getirerek sütun verilerini aşağıda belirtilen yönlere göre yeniden adlandırıyoruz.

In [ ]:
patrol_mapping = {
    'PATROL BORO BKLYN NORTH': 'Brooklyn_N',
    'PATROL BORO BKLYN SOUTH': 'Brooklyn_S',
    'PATROL BORO MAN SOUTH': 'Man_S',
    'PATROL BORO MAN NORTH': 'Man_N',
    'PATROL BORO QUEENS SOUTH': 'Queens_S', 
    'PATROL BORO QUEENS NORTH': 'Queens_N',
    'PATROL BORO BRONX': 'Bronx', 
    'PATROL BORO STATEN ISLAND': 'Staten'
}

# Dictionary kullanarak değerleri değiştiriyoruz
df['patrol'] = df['patrol'].replace(patrol_mapping)

offense_mapping = {
    'Religion/Religious Practice' : 'Religion', 
    'Race/Color' : 'Race',
    'Ethnicity/National Origin/Ancestry' : 'Origin', 
    'Gender' : 'Gender',
    'Sexual Orientation' : 'Orientation', 
    'Disability' : 'Diabled', 
    'Age' : 'Age'
}

# Dictionary kullanarak değerleri değiştiriyoruz
df['offense'] = df['offense'].replace(offense_mapping)
df['motive'] = df['motive'].str.capitalize() #ilk harfleri büyük yapıyoruz
df['county'] = df['county'].str.capitalize()
df['crime'] = df['crime'].str.capitalize()

df.head()

## **5. `Precinct` Verisinin Dizeye Dönüştürülmesi**
Bu, görselleştirme sırasında problemlerden kaçınmak için yapılması gereken bir adımdır.

In [ ]:
df['precinct'] = df['precinct'].astype(str)

In [ ]:
df.info()

## **6. Tarihe göre sıralama**
Burada, verimizi record_date'e göre sıralıyoruz.

In [ ]:
df = df.sort_values(by='record_date').reset_index(drop=True)
df.drop('record_date', axis = 1, inplace = True)

df.head()

In [ ]:
df.head(5)

# **Exploratory Data Analysis(EDA)**
# **Kişisel Veri Analizi**

## **1. Yıllık Trend**
Suç sıklığının yıllara göre `countplot` kullanarak çizilmesi.

In [ ]:
# checking at data
df['year'].value_counts()

In [ ]:
# Her tarihin tekrar sayısını bulma
counts = df['year'].value_counts().sort_index()

# line plot
plt.figure(figsize=(10, 6))
plt.plot(counts.index, counts.values, marker='o', linestyle='-')

# Başlık ve y-ekseni etiketi
plt.title('Crimes Throughout the Years', fontweight='bold')
plt.ylabel('Crime Count', fontweight='bold')
plt.xlabel('Year', fontweight='bold')

# x-ekseni etiketlerini belirleme
plt.xticks([2019, 2020, 2021, 2022, 2023])

# plot
plt.show()

## **2. Aylık Trend**
Yıllar boyunca ``countplot`` kullanarak ``aylar`` boyunca suç sıklığını çizme.

In [ ]:
df['month'].value_counts()

Burada, karışık sıralı aylarla karşılaşacağız, bu yüzden onu düzeltmemiz gerekiyor.

In [ ]:
# defining the correct order of months
month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# 'month' sütununu doğru sırayla kategorik olarak dönüştürüyoruz
df['month'] = pd.Categorical(df['month'], categories = month_order, ordered=True)

# Her ayın tekrar sayısını buluyoruz
counts = df['month'].value_counts().sort_index()

# line-plot
plt.figure(figsize=(10, 6))
plt.plot(counts.index, counts.values, marker='o', linestyle='-')

plt.title('Monthly Crime Trend', fontweight='bold')
plt.ylabel('Crime Count', fontweight='bold')
plt.xlabel('Month', fontweight='bold')

#  plotting
plt.show()

SONUÇ: Suçlar yılın başından ortasına doğru artıyor ve ardından yavaşça azalıyor.

## **3. Yıllara göre Aylık Trend**
Biraz daha derine inmek için, her bir yıl için Aylık Trendi analiz edeceğiz.

In [ ]:
month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

df['month'] = pd.Categorical(df['month'], categories = month_order, ordered = True)

counts = df['month'].value_counts().sort_index()


unique_years = df['year'].unique()

plt.figure(figsize=(16, 8))
for year in unique_years:
    
    year_data = df[df['year'] == year]
    
    year_month_counts = year_data['month'].value_counts().sort_index()
    
    plt.plot(year_month_counts.index, year_month_counts.values, marker='o', linestyle='-', label=str(year))


plt.title('Monthly Crime Trend by Year', fontweight='bold')
plt.ylabel('Crime Count', fontweight='bold')
plt.xlabel('Month', fontweight='bold')


plt.legend(title='Year')


plt.show()


`SONUÇ:` Hemen hemen tüm yılların benzer bir deseni izlediği görülebilir ancak `2021` ve `2022` yılları sırasıyla yılın başında biraz farklıdır ve daha sonra doygunlaşırlar.

## **4. Haftalık Trend**
Suç sıklığını `haftanın` günleri üzerinde bir `countplot` kullanarak çizme.

In [ ]:
df['day'].value_counts()

In [ ]:
day_order = ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']

df['day'] = pd.Categorical(df['day'], categories = day_order, ordered=True)

counts = df['day'].value_counts().sort_index()

# Line plot
plt.figure(figsize=(10, 6))
plt.plot(counts.index, counts.values, marker='o', linestyle='-')

plt.title('Crime Trend throughout the Week', fontweight='bold')
plt.ylabel('Crime Count', fontweight='bold')
plt.xlabel('Day', fontweight='bold')


plt.show()

`SONUÇ:` Suçlar haftanın başında artar ve haftanın sonunda azalır.

## **5. Devriyelerin Suç Sayıları**
Burada, ilgili devriyeler tarafından karşılaşılan ve kaydedilen toplam suç sayısını çiziyoruz.

In [ ]:
df['patrol'].value_counts()

In [ ]:
# Devriyelere göre Suç Sayılarını sayma
counts = df['patrol'].value_counts().sort_index()

num_patrols = len(counts)

colors = sns.color_palette("coolwarm", num_patrols)

# bar plot
plt.figure(figsize=(10, 6))
bars = plt.bar(counts.index, counts.values, color = colors)

plt.title('Crimes encountered by Patrols', fontweight='bold')
plt.ylabel('Crime Count', fontweight='bold')
plt.xlabel('Patrol', fontweight='bold')

plt.ylim(0, 650)

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 5, round(yval), ha='center', va='bottom')

plt.tight_layout()
plt.show()


`SONUÇ:` Dolayısıyla, **Manhattan Güney Devriyesi** en yüksek suç sayısına rastladı, **Staten Island** ise en düşük sayıda suçla karşılaştı.

## **6. İlçeler Boyunca Suçlar**
Suç dağılımını ilçeler boyunca analiz ediyoruz.

In [ ]:
df['county'].value_counts()

In [ ]:
# İlçelere göre suç sayılarını sayma
counts = df['county'].value_counts().sort_index()

num_counties = len(counts)

colors = sns.color_palette("coolwarm", num_counties)

plt.figure(figsize=(10, 6))
bars = plt.bar(counts.index, counts.values, color=colors)
plt.title('Crimes Throughout the Counties', fontweight = 'bold')
plt.ylabel('Crime Count', fontweight = 'bold')
plt.xlabel('Counties', fontweight = 'bold')

plt.ylim(0, 1000)

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 5, round(yval), ha='center', va='bottom')

plt.tight_layout()
plt.show()

`SONUÇ:` `New York` ve `Kings` ilçelerinde kaydedilen suçların sayısı en yüksektir.

## **7. Kategoriye Göre Suç Dağılımı**
Suç dağılımını `kategorilere` göre analiz ediyoruz.

In [ ]:
df['crime'].value_counts()

In [ ]:
counts = df['crime'].value_counts().sort_index()
num_counties = len(counts)
colors = sns.color_palette("coolwarm", num_counties)

plt.figure(figsize=(10, 6))
bars = plt.bar(counts.index, counts.values, color=colors)

plt.title('Crimes Category Distribution', fontweight = 'bold')
plt.ylabel('Crime Count', fontweight = 'bold')
plt.xlabel('Category', fontweight = 'bold')
plt.ylim(0, 1400)

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 5, round(yval), ha='center', va='bottom')

plt.tight_layout()
plt.show()

`SONUÇ:` En yaygın suçlar `Felony` ve `Misdemeanor`dir.

## **8. Motive Göre Suç Dağılımı**
Burada, suçun `motive`ine göre dağılımını görüyoruz.

In [ ]:
df['motive'].value_counts()

In [ ]:
counts = df['motive'].value_counts()
counts.head(5)

In [ ]:
counts = df['motive'].value_counts()
top_5_counts = counts.head(5)

colors = sns.color_palette("coolwarm", len(top_5_counts))

plt.figure(figsize=(10, 6))
bars = plt.bar(top_5_counts.index, top_5_counts.values, color=colors)

plt.title('Top 5 Motive of Crime', fontweight = 'bold')
plt.ylabel('Crime Count', fontweight = 'bold')
plt.xlabel('Motive', fontweight = 'bold')

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 5, round(yval), ha='center', va='bottom')

plt.tight_layout()
plt.show()

`SONUÇ:` `Anti-jewish` suçun en yaygın nedenidir.

## **9. Suç Türüne Göre Suç Dağılımı**
Burada, suçun `suç` türüne göre dağılımını görüyoruz.

In [ ]:
counts = df['offense'].value_counts().sort_index()
num_counties = len(counts)
colors = sns.color_palette("coolwarm", num_counties)

plt.figure(figsize=(10, 6))
bars = plt.bar(counts.index, counts.values, color=colors)

plt.title('Crimes by Offense', fontweight = 'bold')
plt.ylabel('Crime Count', fontweight = 'bold')
plt.xlabel('Offense', fontweight = 'bold')
plt.ylim(0, 1300)

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 5, round(yval), ha='center', va='bottom')

plt.tight_layout()
plt.show()

`SONUÇ:` `Din` suçun en yaygın sebebidir.

## **10. En Yaygın Karakol**
Suçun `Karakollar` boyunca dağılımını analiz ediyoruz.

In [ ]:
df['precinct'].value_counts()

In [ ]:
top_precincts = df['precinct'].value_counts()
top_precincts.head(5)

In [ ]:
counts = df['precinct'].value_counts()
top_5_counts = counts.head(5)

colors = sns.color_palette("coolwarm", len(top_5_counts))

plt.figure(figsize=(10, 6))
bars = plt.bar(top_5_counts.index, top_5_counts.values, color=colors)

plt.title('Top 5 Precincts', fontweight = 'bold')
plt.ylabel('Crime Count', fontweight = 'bold')
plt.xlabel('Precinct', fontweight = 'bold')

plt.ylim(0, 120)

for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval), ha='center', va='bottom')

plt.tight_layout()
plt.show()

`SONUÇ:` Dolayısıyla, `Karakol 90` en fazla suç olayının yaşandığı yer olmuştur.

In [ ]:
df.info()
df.head(5)

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


# 'month' sütununu numerik değerlere dönüştür
month_dict = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
df['month'] = df['month'].map(month_dict)

# Label encoding için sütunları seç
label_encode_columns = ['day', 'patrol', 'county', 'crime', 'motive', 'offense']

# Label encoding yap
label_encoder = LabelEncoder()
for column in label_encode_columns:
    df[column] = label_encoder.fit_transform(df[column])



## **Korelasyon Matrisi Hesaplama**

In [ ]:
correlation_matrix = df.corr()

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Korelasyon Matrisi')
plt.show()

## **Öznitelik Seçimi**

In [ ]:
# Bağımsız değişkenler ve bağımlı değişken ayırma
X = df.drop('precinct', axis=1)
y = df['precinct']

# Öznitelik seçimi yap
best_features = SelectKBest(score_func=chi2, k='all')
fit = best_features.fit(X, y)

# Her özelliğin p-değerlerini ve p-değerlerine göre sıralanmış bir listeyi al sonra Özellikler ve p-değerlerini birleştir
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

feature_scores = pd.concat([dfcolumns, dfscores], axis=1)
feature_scores.columns = ['Feature', 'Score']

# P-değerlerine göre özellikleri sıralama
feature_scores = feature_scores.sort_values(by='Score', ascending=False)

# En iyi 5 özelliği seçme
selected_features = feature_scores.head(5)['Feature'].tolist()
print("En iyi 5 özellik:")
print(selected_features)


## **Eksik Değer Analizi**

In [ ]:
df.isnull().sum()